In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

from tqdm import tqdm

training_folder = r'../'
gis_folder = training_folder + r'gis/corse/'
data_folder = training_folder + r'data/corse/'

# chargement des données

In [3]:
nodes = gpd.read_file(gis_folder + r'osmdrive/nodes/nodes.shp')
stops = gpd.read_file(gis_folder + r'stops_corse.shp')
communes = gpd.read_file(gis_folder + r'communes_corse.shp')

In [4]:
nodes.crs = {'init' :'epsg:4326'}
stops.crs = {'init' :'epsg:4326'}
communes.crs = {'init' :'epsg:4326'}

nodes = nodes.to_crs({'init': 'epsg:2154'}) 
stops = stops.to_crs({'init': 'epsg:2154'}) 
communes = communes.to_crs({'init': 'epsg:2154'}) 

C:\Users\rgoix\AppData\Local\Continuum\anaconda3\envs\quetzal_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\rgoix\AppData\Local\Continuum\anaconda3\envs\quetzal_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\rgoix\AppData\Local\Continuum\anaconda3\envs\quetzal_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is depr

In [5]:
nodes.head(1)

,osmid,highway,geometry
0,939851776,None,POINT (1223345.366 6074305.296)


In [6]:
stops.head(1)

,name,stop,geometry
0,"Gare de Campo dell'Oro (Ajaccio), corse",0,POINT (1181123.516 6110402.106)


In [7]:
communes.head(1)

,insee,geometry
0,2B047,"POLYGON ((1194340.897 6154934.152, 1194481.235..."


# Matrices des distances 
## stop -> node et commune -> node

In [8]:
nodetuples = nodes[['osmid', 'geometry']].values.tolist()
stoptuples = stops[['stop', 'geometry']].values.tolist()
communetuples = communes[['insee', 'geometry']].values.tolist()

## stop node

In [9]:
stop_node_distances = []

for stop_id, stop_geometry in tqdm(stoptuples):
    for node_id, node_geometry in nodetuples:
        distance = node_geometry.distance(stop_geometry)
        stop_node_distances.append((node_id, stop_id, distance))
        
# on transforme les listes de tuples en DataFrames
stop_node_distance_dataframe = pd.DataFrame(
    stop_node_distances, 
    columns = ['stop_node', 'stop', 'distance']
)

stop_node_distance_dataframe.sort_values(by='distance', inplace=True)

100%|██████████| 50/50 [00:03<00:00, 15.50it/s]


In [13]:
len(stops) * len(nodes)

848300

In [10]:
len(stop_node_distance_dataframe)

848300

In [14]:
stop_node_distance_dataframe.head()

,stop_node,stop,distance
202262,1438772679,11,7.085744
526269,938869755,35,17.101087
351879,4460494743,22,17.591249
734776,2313442539,54,22.884806
446354,2313442539,28,22.884806


## commune node
On fait la même chose avec les communes

In [15]:
commune_node_distances = []
        
for commune_id, commune_geometry in tqdm(communetuples):
    for node_id, node_geometry in nodetuples:
        distance = node_geometry.distance(commune_geometry)
        commune_node_distances.append((node_id, commune_id, distance))
        
commune_node_distance_dataframe = pd.DataFrame(
    commune_node_distances, 
    columns = ['commune_node', 'commune', 'distance']
)

commune_node_distance_dataframe.sort_values(by='distance', inplace=True)

commune_node_distance_dataframe.head() # quand le nœud est dans la commune, la distance est nulle

100%|██████████| 360/360 [00:37<00:00,  9.57it/s]


,commune_node,commune,distance
4393951,1797913766,2B037,0.0
60235,6286905402,2B341,0.0
4759110,338462238,2B207,0.0
2296602,1900438245,2A130,0.0
3441934,386850990,2A041,0.0


In [16]:
# la table est assez large, on va la stocker au format hdf, plus rapide et leger que CSV
len(commune_node_distance_dataframe)

6107760

In [17]:
def top(df, n=5, column='data'):
    return df.sort_values(by=column)[:n]
commune_node_distance_dataframe = commune_node_distance_dataframe.groupby('commune').apply(top, n=20, column='distance')

In [18]:
commune_node_distance_dataframe.to_hdf(data_folder + 'distance_to_node.hdf', 'communes', index=False, mode='w')
stop_node_distance_dataframe.to_hdf(data_folder + 'distance_to_node.hdf', 'stops', index=False, mode='a')